In [1]:
import findspark

findspark.init()

In [2]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, SparkTrials, Trials

In [ ]:
import pandas as pd
import numpy as np
import pickle

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, SparkTrials, Trials
import mlflow
from mlflow.tracking import MlflowClient

import os
from loguru import logger
from functools import partial
from argparse import ArgumentParser
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import roc_auc_score,fbeta_score,classification_report
from sklearn.metrics import log_loss,roc_curve,auc,average_precision_score,accuracy_score,r2_score

from catboost import CatBoostClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.inspection import permutation_importance


from pyspark.sql import SparkSession, DataFrame, functions as F
from pyspark.sql.types import IntegerType,LongType,DoubleType,StringType

from datetime import datetime, timedelta,timezone


/opt/conda/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.2.3) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [4]:
import bit_functions
import config_btc
import classes

/home/ubuntu/classes.py:9: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [5]:
bucket_name = 'cold-s3-bucket'

input_path = f"s3a://{bucket_name}/btc_prepaired_data.parquet"

In [6]:
def get_spark():

    spark = (
            SparkSession
            .builder
            .appName("OTUS")
            .getOrCreate()
            )
    return spark

In [ ]:
spark = get_spark()


In [8]:
row_sdf = spark.read.parquet(input_path)

In [10]:
clusters_selected_cols_08 = ['MA10_REL_shift6_cross',
 'EMA100_REL',
 'EMA60_REL_shift12_cross',
 'MACDHist_REL_shift7',
 'Close_15_to_MA15',
 'MA10_REL_shift1_cross',
 'MA15_REL_shift1',
 'MA100_REL_shift12',
 'EMA200_REL_shift12_cross',
 'Close_9_to_10',
 'MACDHist_REL',
 'EMA32_REL_shift1',
 'RSI9_up_70_perman',
 'EMA20_REL_shift6_cross',
 'MA10_REL_shift6',
 'EMA26_REL_shift24_cross',
 'EMA75_REL_shift24',
 'EMA200_REL_shift24_cross',
 'RSI9_dw_20_perman',
 'EMA75_REL_shift6',
 'EMA75_REL_shift1_cross',
 'MA20_REL_shift12_cross',
 'EMA12_REL_shift12',
 'ROC200sm',
 'EMA32_REL_shift6',
 'MA75_REL_shift6_cross',
 'MACDHist_corr_roll_mean3_shift1',
 'Close_10_to_MA15',
 'Close_0_to_4',
 'RSI14_up_80',
 'MA150_REL_shift1',
 'EMA26_REL',
 'EMA26_REL_shift12',
 'std_20_rel',
 'Close_4_to_MA15',
 'RSI14_dw_30',
 'EMA20_REL_shift24',
 'Close_3_to_4',
 'MA50_REL_shift24_cross',
 'EMA14_REL_shift24_cross',
 'MA10_REL_shift1',
 'AweOs_cor_roll_mean3',
 'AweOs_corr_1_to_3',
 'MA150_REL_shift6_cross',
 'RSI20_up_70_perman',
 'MACDHist_corr_roll_mean3_shift6',
 'Close_5_to_MA15',
 'EMA150_REL_shift6',
 'RSI27_dw_20',
 'MA10_REL_shift12_cross',
 'EMA38_REL_shift6',
 'EMA32_REL_shift1_cross',
 'RSI20_up_80',
 'Close_m3_2_to_MA15',
 'Close_m3_6_to_MA15',
 'RSI27_dw_30',
 'AweOs_corr_3_to_6',
 'EMA32_REL_shift12_cross',
 'MA5_REL_shift24_cross',
 'EMA12_REL',
 'MACDHist_corr_1_to_3',
 'EMA44_REL_shift6_cross',
 'MACDHist_corr',
 'AweOs_cor_roll_mean3_shift6',
 'level_close_act_w',
 'RSI14_dw_20_perman',
 'MA5_REL_shift24',
 'MA20_REL_shift6',
 'Close_6_to_MA15',
 'EMA38_REL_shift12',
 'MA5_REL_shift12',
 'Close_8_to_MA15',
 'EMA100_REL_shift24',
 'EMA8_REL_shift24_cross',
 'MA5_REL_shift6_cross',
 'Close_7_to_8',
 'MA20_REL_shift1',
 'MA5_REL_shift12_cross',
 'EMA_gor_1_REL_cross_all',
 'RSI20_dw_20_perman',
 'AweOs_cor',
 'MA200_REL_shift12',
 'RSI9_up_80',
 'MACDHist_REL_shift3',
 'EMA200_REL_shift1_cross',
 'RSI9_dw_30_perman',
 'EMA44_REL_shift24',
 'MA5_REL',
 'MA100_REL_shift12_cross',
 'EMA8_REL',
 'RSI14_up_70_perman',
 'MA10_REL_shift24',
 'Close_13_to_MA15',
 'MACDHist_corr_3_to_6',
 'MA5_REL_shift6',
 'RSI27_up_70_perman',
 'MA_gor_6_REL_cross_all',
 'EMA100_REL_shift24_cross',
 'EMA14_REL_shift6_cross',
 'RSI20_dw_30',
 'Close_2_to_3',
 'EMA20_REL_shift1_cross',
 'RSI27_up_80',
 'MACDHist_corr_roll_mean3_shift3',
 'Close_5_to_6',
 'Close_6_to_7',
 'MACDHist_corr_roll_mean3',
 'AweOs_corr_0_to_6',
 'MA25_REL_shift1',
 'Close_2_to_MA15',
 'EMA12_REL_shift6',
 'Close_0_to_3',
 'step_to_fiba',
 'Close_9_to_MA15',
 'EMA200_REL_shift24',
 'AweOs_corr_0_to_1',
 'MA_gor_24_REL_cross_all',
 'Close_1_to_2',
 'Close_0_to_6',
 'MA50_REL_shift1',
 'EMA14_REL_shift12',
 'MA20_REL',
 'MA15_REL_shift1_cross',
 'MACDHist_corr_0_to_1',
 'EMA_gor_12_REL_cross_all',
 'MACDHist_corr_0_to_6',
 'Close_8_to_9',
 'Close_11_to_12',
 'EMA12_REL_shift12_cross',
 'EMA50_REL_shift24_cross',
 'MA15_REL',
 'Close_4_to_5',
 'EMA50_REL_shift1_cross',
 'EMA44_REL',
 'Close_0_to_1',
 'MA150_REL_shift1_cross',
 'Close_0_to_12',
 'MA100_REL_shift6_cross',
 'EMA200_REL_shift6_cross',
 'EMA44_REL_shift12_cross',
 'MA100_REL_shift1_cross',
 'MACDHist_REL_shift10',
 'level_close_old_w',
 'EMA75_REL_shift12',
 'std_100_rel',
 'MA75_REL_shift1_cross',
 'EMA20_REL',
 'Close_10_to_11',
 'MA75_REL_shift1',
 'AweOs_cor_shift3',
 'EMA12_REL_shift1',
 'MA75_REL_shift12_cross',
 'MA10_REL_shift12',
 'MA200_REL',
 'RSI14',
 'MA5_REL_shift1_cross',
 'MA75_REL_shift24_cross',
 'EMA26_REL_shift1',
 'MA150_REL_shift12_cross',
 'EMA12_REL_shift24',
 'EMA150_REL_shift1_cross',
 'EMA60_REL_shift6_cross',
 'EMA150_REL_shift6_cross',
 'Close_coef_nak',
 'TSI',
 'MA200_REL_shift1_cross',
 'Close_12_to_MA15',
 'EMA100_REL_shift6',
 'MA25_REL_shift6',
 'Close_11_to_MA15',
 'RSI20',
 'Close_m3_4_to_MA15',
 'Close_0_to_9',
 'RSI27',
 'EMA26_REL_shift6_cross',
 'RSI9',
 'EMA12_REL_shift1_cross',
 'MA25_REL_shift1_cross',
 'EMA60_REL_shift1',
 'MA100_REL',
 'AweOs_cor_roll_mean3_shift1',
 'MA50_REL_shift1_cross',
 'MACDHist_corr_shift3',
 'EMA32_REL',
 'EMA100_REL_shift1_cross',
 'EMA100_REL_shift12_cross',
 'MA25_REL',
 'MACDHist_REL_shift5',
 'MA150_REL_shift24_cross',
 'EMA100_REL_shift6_cross',
 'MACDHist_corr_shift6',
 'MA50_REL_shift12_cross',
 'MA100_REL_shift24_cross',
 'MA200_REL_shift6',
 'MA200_REL_shift6_cross',
 'MA200_REL_shift12_cross',
 'EMA50_REL_shift6',
 'EMA150_REL',
 'EMA20_REL_shift12',
 'EMA44_REL_shift24_cross',
 'EMA150_REL_shift24',
 'MA15_REL_shift6_cross',
 'Close_m3_3_to_MA15',
 'MA25_REL_shift6_cross',
 'EMA75_REL_shift24_cross',
 'MA50_REL_shift6_cross',
 'MA25_REL_shift24_cross',
 'Close_7_to_MA15',
 'EMA20_REL_shift1',
 'MA20_REL_shift6_cross',
 'MA_gor_1_REL_cross_all',
 'EMA50_REL_shift12',
 'Close_m3_9_to_MA15',
 'Close_3_to_MA15',
 'MA20_REL_shift1_cross',
 'Close_0_to_5',
 'MA200_REL_shift24_cross',
 'AweOs_cor_roll_mean3_shift3',
 'EMA75_REL_shift1',
 'MA25_REL_shift12_cross',
 'EMA200_REL',
 'MA150_REL_shift12',
 'MACDHist_REL_shift1',
 'EMA150_REL_shift24_cross',
 'EMA50_REL_shift1',
 'MA50_REL',
 'EMA_gor_6_REL_cross_all',
 'EMA75_REL_shift6_cross',
 'AweOs_cor_shift6',
 'EMA75_REL_shift12_cross',
 'EMA150_REL_shift12_cross',
 'EMA32_REL_shift24',
 'EMA20_REL_shift12_cross',
 'MA15_REL_shift24_cross',
 'Close_m3_0_to_MA15',
 'MA10_REL',
 'MA20_REL_shift24_cross',
 'MA15_REL_shift12_cross',
 'Close_14_to_MA15',
 'Close_m3_5_to_MA15',
 'MA_gor_12_REL_cross_all',
 'EMA60_REL',
 'EMA60_REL_shift1_cross']

In [15]:
data_set_4h

,MA10_REL_shift6_cross,EMA100_REL,EMA60_REL_shift12_cross,MACDHist_REL_shift7,Close_15_to_MA15,MA10_REL_shift1_cross,MA15_REL_shift1,MA100_REL_shift12,EMA200_REL_shift12_cross,Close_9_to_10,...,Close_14_to_MA15,Close_m3_5_to_MA15,MA_gor_12_REL_cross_all,EMA60_REL,EMA60_REL_shift1_cross,target_new,ROWNUM,ROW_NUM,target_new_up,target_new_dw
0,0,1.000000,0,NaN,1.000000,0,NaN,NaN,0,1.000000,...,1.000000,NaN,0.0,1.000000,0,-1,1,0,0,1
1,0,0.982884,0,NaN,1.000000,-1,1.000000,NaN,0,1.000000,...,1.000000,NaN,0.0,0.983110,-1,-1,2,1,0,1
2,0,0.999816,0,NaN,1.000000,1,0.991269,NaN,0,1.000000,...,1.000000,NaN,0.0,1.000041,1,-1,3,2,0,1
3,0,1.005962,0,NaN,1.000000,0,1.005561,NaN,0,1.000000,...,1.000000,NaN,0.0,1.006102,0,-1,4,3,0,1
4,0,1.015083,0,NaN,1.000000,0,1.008898,NaN,0,1.000000,...,1.000000,NaN,0.0,1.015020,0,-1,5,4,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8499,1,1.019871,0,-0.949283,0.974989,1,0.987738,1.008096,0,0.976845,...,0.981005,0.999784,-0.1,1.013543,1,0,8500,8499,0,0
8500,1,1.036180,0,-0.679630,0.981540,0,1.011048,1.000694,0,0.999270,...,0.997090,1.007055,0.1,1.029478,0,0,8501,8500,0,0
8501,1,1.061040,1,0.071555,1.000267,0,1.027380,0.973010,0,0.999341,...,0.987724,1.016206,0.8,1.053586,0,0,8502,8501,0,0
8502,1,1.053187,0,1.827821,0.989646,0,1.049905,0.995473,0,1.005880,...,0.996418,1.018987,0.2,1.045319,0,0,8503,8502,0,0


In [19]:
data_set_4h = row_sdf.toPandas()

data_set_4h['target_new_up'] = np.where(data_set_4h['target_new']==1,1,0)
data_set_4h['target_new_dw'] = np.where(data_set_4h['target_new']==-1,1,0)

print('размер дата сета',data_set_4h.shape)

cols_up_new = bit_functions.permutaion_list(data_set_4h,'target_new_up',clusters_selected_cols_08)

cols_dw_new = bit_functions.permutaion_list(data_set_4h,'target_new_dw',clusters_selected_cols_08)


final_columns_list = list(set(cols_up_new + cols_dw_new))

print('длинна фичей финал',len(final_columns_list))


X = data_set_4h[(data_set_4h.ROWNUM>=300)&(data_set_4h.ROWNUM<8500)][final_columns_list].copy()
y = data_set_4h[(data_set_4h.ROWNUM>=300)&(data_set_4h.ROWNUM<8500)].target_new

размер дата сета (8504, 248)
Количество метрик 244
first_itter:  115
second_itter:  77
Количество метрик 244
first_itter:  243


ValueError: min() arg is an empty sequence

In [10]:
data_set_4h = row_sdf.toPandas()

data_set_4h['target_new_up'] = np.where(data_set_4h['target_new']==1,1,0)
data_set_4h['target_new_dw'] = np.where(data_set_4h['target_new']==-1,1,0)

# print('размер дата сета',data_set_4h.shape)

# cols_up_new = bit_functions.permutaion_list(data_set_4h,'target_new_up',clusters_selected_cols_08)

# cols_dw_new = bit_functions.permutaion_list(data_set_4h,'target_new_dw',clusters_selected_cols_08)


# final_columns_list = list(set(cols_up_new + cols_dw_new))

# print('длинна фичей финал',len(final_columns_list))


X = data_set_4h[(data_set_4h.ROWNUM>=300)&(data_set_4h.ROWNUM<8500)][final_columns_list].copy()
y = data_set_4h[(data_set_4h.ROWNUM>=300)&(data_set_4h.ROWNUM<8500)].target_new

In [9]:
final_columns_list = ['MA5_REL',
 'EMA8_REL',
 'std_100_rel',
 'MA10_REL',
 'TSI',
 'MA150_REL_shift1',
 'Close_0_to_4',
 'Close_0_to_5',
 'EMA12_REL',
 'std_20_rel',
 'RSI14_up_70_perman',
 'RSI27',
 'AweOs_corr_0_to_6',
 'MA200_REL_shift6',
 'EMA150_REL',
 'MACDHist_corr',
 'Close_m3_5_to_MA15',
 'MA200_REL',
 'RSI27_up_70_perman',
 'ROC200sm',
 'MACDHist_corr_1_to_3',
 'Close_4_to_MA15',
 'Close_5_to_MA15',
 'MACDHist_corr_0_to_6',
 'MA50_REL',
 'RSI20',
 'EMA32_REL_shift24',
 'EMA14_REL_shift12',
 'MACDHist_corr_0_to_1',
 'Close_1_to_2',
 'EMA50_REL_shift1',
 'EMA100_REL',
 'RSI20_up_70_perman',
 'MA25_REL_shift6',
 'AweOs_cor',
 'step_to_fiba',
 'Close_0_to_1',
 'MA20_REL',
 'MA15_REL',
 'Close_m3_3_to_MA15',
 'AweOs_cor_roll_mean3_shift6',
 'EMA150_REL_shift6',
 'Close_m3_2_to_MA15',
 'MA150_REL_shift12',
 'EMA26_REL',
 'MACDHist_REL_shift5',
 'EMA20_REL_shift12',
 'EMA60_REL',
 'Close_8_to_MA15',
 'AweOs_corr_0_to_1',
 'EMA75_REL_shift12',
 'Close_0_to_3',
 'MA15_REL_shift6_cross',
 'EMA_gor_12_REL_cross_all',
 'MA100_REL',
 'EMA12_REL_shift1',
 'EMA12_REL_shift12_cross',
 'MA15_REL_shift1',
 'AweOs_cor_shift6',
 'Close_0_to_6',
 'EMA32_REL',
 'Close_8_to_9',
 'Close_7_to_8',
 'MA200_REL_shift12',
 'EMA12_REL_shift6',
 'EMA44_REL',
 'MA15_REL_shift12_cross',
 'Close_11_to_MA15',
 'MA50_REL_shift12_cross',
 'EMA100_REL_shift6',
 'MA25_REL_shift1',
 'EMA200_REL_shift24',
 'RSI27_dw_30',
 'MA5_REL_shift6_cross',
 'MACDHist_REL_shift3',
 'EMA26_REL_shift1',
 'EMA20_REL']

In [11]:
X = data_set_4h[(data_set_4h.ROWNUM>=300)&(data_set_4h.ROWNUM<8500)][final_columns_list].copy()
y = data_set_4h[(data_set_4h.ROWNUM>=300)&(data_set_4h.ROWNUM<8500)].target_new

In [12]:
mlflow_ip = '158.160.45.112'

os.environ['MLFLOW_S3_ENDPOINT_URL'] = 'https://storage.yandexcloud.net'
os.environ['MLFLOW_TRACKING_URI'] = f'http://{mlflow_ip}:8000'

In [24]:
mlflow.set_experiment('classification')

<Experiment: artifact_location='s3://airflow-bucket-3db29db4338f25d9/artifacts/1', creation_time=1741710726344, experiment_id='1', last_update_time=1741710726344, lifecycle_stage='active', name='classification', tags={}>

In [ ]:
def objective(params,pipeline,X_train, y_train):
    """
    Кросс-валидация с текущими гиперпараметрами

    :params: гиперпараметры
    :pipeline: модель
    :X_train: матрица признаков
    :y_train: вектор меток объектов
    :return: средняя точность на кросс-валидации
    """

    logger.info(f"params {params}")

    # задаём модели требуемые параметры
    pipeline.set_params(**params)

    ovr = OneVsRestClassifier(estimator=pipeline)
    cv=KFold(n_splits=3,shuffle=False)


    score=np.mean(cross_val_score(ovr, X_train, y_train, cv=cv,scoring = 'roc_auc_ovr_weighted'))
    cv_f1_w=np.mean(cross_val_score(ovr,  X_train, y_train, cv=cv,scoring = 'f1_weighted'))

    dct_metrics = {
        'auc': score,
        'f1': cv_f1_w,
        }

    with mlflow.start_run():
        mlflow.log_params(params)
        mlflow.log_metrics(dct_metrics)

    # возвращаем результаты, которые записываются в Trials()
    return   {'loss': -score, 'params': params, 'status': STATUS_OK, 'model': ovr}

In [ ]:
search_space = {

                'n_estimators' : hp.choice(label='N_E',
                        options=np.arange(100, 1000,100, dtype=int)),

                'learning_rate' : hp.uniform(label='L_R',
                        low=0.01,
                        high=0.1),

                'l2_leaf_reg': hp.uniform(label='R_L',
                        low=1,
                        high=500),
                }

In [ ]:
# запускаем hyperopt
trials = Trials()
model =CatBoostClassifier(verbose=False)
best = fmin(
          # функция для оптимизации
            fn=partial(objective, pipeline=model, X_train=X, y_train=y),
            #fn=objective(params,X_train, y_train)
          # пространство поиска гиперпараметров
            space=search_space,
          # алгоритм поиска
            algo=tpe.suggest,
          # число итераций
          # (можно ещё указать и время поиска)
            max_evals=2,
          # куда сохранять историю поиска
            trials=trials,
          # random state
            #rstate=np.random.RandomState(1),
          # progressbar
            show_progressbar=True
        )


  0%|          | 0/2 [00:00<?, ?trial/s, best loss=?]

2025-03-11 16:58:47.373 | INFO     | __main__:objective:12 - params {'l2_leaf_reg': 77.6286261338893, 'learning_rate': 0.02727657376247558, 'n_estimators': 700}
2025/03/11 17:02:03 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh(<full-path-to-git-executable>)

All git commands will error until this is rectified.

This initial message can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|silent|none|n|0: for no message or exception
    - warn|w|warning|log|l|1: for a warning message (logging level CRITICAL, displayed by default)
    - error|e|exception|raise|r|2: for a raised 


 50%|█████     | 1/2 [03:16<03:16, 196.49s/trial, best loss: -0.7765718976733322]

2025-03-11 17:02:03.863 | INFO     | __main__:objective:12 - params {'l2_leaf_reg': 145.03065884192517, 'learning_rate': 0.042481044752712444, 'n_estimators': 300}
2025/03/11 17:03:29 INFO mlflow.tracking._tracking_service.client: 🏃 View run hilarious-fox-579 at: http://158.160.45.112:8000/#/experiments/1/runs/4910640586bb4708a83d029f176adee5.

2025/03/11 17:03:29 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://158.160.45.112:8000/#/experiments/1.



100%|██████████| 2/2 [04:42<00:00, 141.13s/trial, best loss: -0.7846956746453363]


In [28]:
best_result = trials.results[np.argmin([r['loss'] for r in trials.results])]['loss']
best_params = trials.results[np.argmin([r['loss'] for r in trials.results])]['params']

print('готово')
print(best_result)
print(best_params)

готово
-0.7846956746453363
{'l2_leaf_reg': 145.03065884192517, 'learning_rate': 0.042481044752712444, 'n_estimators': 300}


In [ ]:
S3_ENDPOINT_URL=https://storage.yandexcloud.net
S3_BUCKET_NAME=
S3_ACCESS_KEY=
S3_SECRET_KEY=

In [ ]:

bucket_art = 'airflow-bucket-3db29db4338f25d9'

os.environ["AWS_ACCESS_KEY_ID"] = f'{aws_acc}'
os.environ["AWS_SECRET_ACCESS_KEY"] = f'{aws_sec}'
os.environ["S3_ENDPOINT_URL"] = 'https://storage.yandexcloud.net'
os.environ["S3_BUCKET_NAME"] = bucket_art

In [ ]:
model_name = 'classification'

    #experiment_id = get_experiment_id(model_name)

with mlflow.start_run() as run:


    model =CatBoostClassifier(verbose=False)
    #model.set_params(**best_params)
    model_best = OneVsRestClassifier(estimator=model)
    model_best.fit(X,y)


    mlflow.sklearn.log_model(model_best, artifact_path="models", registered_model_name=model_name)

    # run_id = run.info.run_id

    mlflow.log_params(best_params)
    mlflow.log_metric('auc', -best_result)

    client = MlflowClient()
    model_versions = client.search_model_versions(filter_string=f"name = '{model_name}'")


    if len(model_versions)==1:
        cur_version = model_versions[0].version
        client.transition_model_version_stage(name=model_name, version=cur_version, stage="Production")
    else:
        cur_version = client.get_latest_versions(model_name, stages=["None"])
        client.transition_model_version_stage(name=model_name, version=cur_version, stage="Staging")


    mlflow.end_run()

2025/03/11 18:49:06 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/03/11 18:49:06 INFO mlflow.tracking._tracking_service.client: 🏃 View run funny-wasp-164 at: http://158.160.45.112:8000/#/experiments/0/runs/00460b21494446dab92431233e33f62b.
2025/03/11 18:49:06 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://158.160.45.112:8000/#/experiments/0.


ImportError: cannot import name 'DEFAULT_CIPHERS' from 'urllib3.util.ssl_' (/home/ubuntu/.local/lib/python3.8/site-packages/urllib3/util/ssl_.py)

In [60]:
!pip install openai requests==2.29.0 urllib3==1.26.20

Defaulting to user installation because normal site-packages is not writeable
  Using cached urllib3-1.26.20-py2.py3-none-any.whl (144 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.16
    Uninstalling urllib3-1.26.16:
      Successfully uninstalled urllib3-1.26.16


In [ ]:
mlflow==1.30.0
boto3==1.24.59

In [ ]:
urllib3==

In [75]:
!pip install --upgrade pip

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 1.8 MB 1.3 MB/s eta 0:00:01


In [78]:
# !pip install boto3==1.18.10
# !pip install mlflow==1.19.0
!pip install boto3==1.16.7
!pip install botocore==1.19.63
!pip install mlflow==2.17.2
!pip install requests==2.32.3
#!pip install urllib3==1.26.20
!pip install urllib3==1.25.11


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 22.8 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: botocore
    Found existing installation: botocore 1.19.7
    Uninstalling botocore-1.19.7:
      Successfully uninstalled botocore-1.19.7
Defaulting to user installation because normal site-packages is not writeable
  Using cached urllib3-2.2.3-py3-none-any.whl.metadata (6.5 kB)


Using cached urllib3-2.2.3-py3-none-any.whl (126 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.25.11
    Uninstalling urllib3-1.25.11:
      Successfully uninstalled urllib3-1.25.11
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
botocore 1.19.63 requires urllib3<1.27,>=1.25.4; python_version != "3.4", but you have urllib3 2.2.3 which is incompatible.
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Using cached urllib3-1.25.11-py2.py3-none-any.whl.metadata (41 kB)
Using cached urllib3-1.25.11-py2.py3-none-any.whl (127 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.2.3
    Uninstalling urllib3-2.2.3:
      Successfully uninstalled urllib3-2.2.3
ERROR: pip's dependency resolver does not curr

In [ ]:
requests==2.26.0
starlette-exporter==0.10.0
mlflow==1.19.0
boto3==1.18.10

In [66]:
!pip show urllib3

Name: urllib3
Version: 1.26.20
Summary: HTTP library with thread-safe connection pooling, file post, and more.
Home-page: https://urllib3.readthedocs.io/
Author: Andrey Petrov
Author-email: andrey.petrov@shazow.net
License: MIT
Location: /home/ubuntu/.local/lib/python3.8/site-packages
Requires: 
Required-by: requests, docker, databricks-cli, botocore


In [67]:
!pip show requests

Name: requests
Version: 2.26.0
Summary: Python HTTP for Humans.
Home-page: https://requests.readthedocs.io
Author: Kenneth Reitz
Author-email: me@kennethreitz.org
License: Apache 2.0
Location: /home/ubuntu/.local/lib/python3.8/site-packages
Requires: urllib3, certifi, charset-normalizer, idna
Required-by: mlflow, mlflow-skinny, docker, databricks-sdk, databricks-cli, yandexcloud, jupyterlab-server, conda, conda-package-streaming


In [68]:
!pip show botocore

Name: botocore
Version: 1.21.65
Summary: Low-level, data-driven core of boto 3.
Home-page: https://github.com/boto/botocore
Author: Amazon Web Services
Author-email: None
License: Apache License 2.0
Location: /home/ubuntu/.local/lib/python3.8/site-packages
Requires: urllib3, jmespath, python-dateutil
Required-by: s3transfer, boto3


In [43]:
!pip show boto3

Name: boto3
Version: 1.37.10
Summary: The AWS SDK for Python
Home-page: https://github.com/boto/boto3
Author: Amazon Web Services
Author-email: None
License: Apache License 2.0
Location: /home/ubuntu/.local/lib/python3.8/site-packages
Requires: jmespath, botocore, s3transfer
Required-by: 


In [ ]:
boto3==1.34.106

In [ ]:
mlflow==2.12.2

Defaulting to user installation because normal site-packages is not writeable


In [50]:
!pip install boto3==1.34.106

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 139 kB 1.3 MB/s eta 0:00:01
     |████████████████████████████████| 12.5 MB 8.0 MB/s eta 0:00:01
     |████████████████████████████████| 83 kB 2.7 MB/s  eta 0:00:01
  Attempting uninstall: botocore
    Found existing installation: botocore 1.37.10
    Uninstalling botocore-1.37.10:
      Successfully uninstalled botocore-1.37.10
  Attempting uninstall: s3transfer
    Found existing installation: s3transfer 0.11.4
    Uninstalling s3transfer-0.11.4:
      Successfully uninstalled s3transfer-0.11.4
  Attempting uninstall: boto3
    Found existing installation: boto3 1.37.10
    Uninstalling boto3-1.37.10:
      Successfully uninstalled boto3-1.37.10


In [47]:
!pip install mlflow==2.12.2

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 20.2 MB 1.3 MB/s eta 0:00:01
     |████████████████████████████████| 38.5 MB 22 kB/s s eta 0:00:01     |█████████████▍                  | 16.1 MB 48.6 MB/s eta 0:00:01
     |████████████████████████████████| 84 kB 323 kB/s  eta 0:00:01
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 17.0.0
    Uninstalling pyarrow-17.0.0:
      Successfully uninstalled pyarrow-17.0.0
  Attempting uninstall: gunicorn
    Found existing installation: gunicorn 23.0.0
    Uninstalling gunicorn-23.0.0:
      Successfully uninstalled gunicorn-23.0.0
  Attempting uninstall: mlflow
    Found existing installation: mlflow 2.17.2
    Uninstalling mlflow-2.17.2:
      Successfully uninstalled mlflow-2.17.2


In [ ]:
mlflow.catboost.log_model(model_2, model_name)

In [ ]:
mlflow.sklearn.log_model(model, "one_vs_rest_classifier")

In [33]:
!pip show urllib3

Name: urllib3
Version: 1.26.20
Summary: HTTP library with thread-safe connection pooling, file post, and more.
Home-page: https://urllib3.readthedocs.io/
Author: Andrey Petrov
Author-email: andrey.petrov@shazow.net
License: MIT
Location: /home/ubuntu/.local/lib/python3.8/site-packages
Requires: 
Required-by: requests, docker, botocore


In [ ]:
requests==2.28.2

In [45]:
!pip install requests==2.30.0

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 62 kB 786 kB/s eta 0:00:011
  Attempting uninstall: requests
    Found existing installation: requests 2.28.2
    Uninstalling requests-2.28.2:
      Successfully uninstalled requests-2.28.2


In [35]:
!pip show requests

Name: requests
Version: 2.32.3
Summary: Python HTTP for Humans.
Home-page: https://requests.readthedocs.io
Author: Kenneth Reitz
Author-email: me@kennethreitz.org
License: Apache-2.0
Location: /home/ubuntu/.local/lib/python3.8/site-packages
Requires: certifi, idna, urllib3, charset-normalizer
Required-by: mlflow-skinny, docker, databricks-sdk, yandexcloud, jupyterlab-server, conda, conda-package-streaming


Defaulting to user installation because normal site-packages is not writeable
  Using cached urllib3-1.26.20-py2.py3-none-any.whl (144 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.2.3
    Uninstalling urllib3-2.2.3:
      Successfully uninstalled urllib3-2.2.3


In [30]:
!pip show mlflow

Name: mlflow
Version: 2.17.2
Summary: MLflow is an open source platform for the complete machine learning lifecycle
Home-page: None
Author: None
Author-email: None
License: Copyright 2018 Databricks, Inc.  All rights reserved.
        
        				Apache License
                                   Version 2.0, January 2004
                                http://www.apache.org/licenses/
        
           TERMS AND CONDITIONS FOR USE, REPRODUCTION, AND DISTRIBUTION
        
           1. Definitions.
        
              "License" shall mean the terms and conditions for use, reproduction,
              and distribution as defined by Sections 1 through 9 of this document.
        
              "Licensor" shall mean the copyright owner or entity authorized by
              the copyright owner that is granting the License.
        
              "Legal Entity" shall mean the union of the acting entity and all
              other entities that control, are controlled by, or are under com

In [17]:
!pip install --upgrade urllib3

Defaulting to user installation because normal site-packages is not writeable
Requirement already up-to-date: urllib3 in ./.local/lib/python3.8/site-packages (2.2.3)


Name: botocore
Version: 1.37.10
Summary: Low-level, data-driven core of boto 3.
Home-page: https://github.com/boto/botocore
Author: Amazon Web Services
Author-email: None
License: Apache License 2.0
Location: /home/ubuntu/.local/lib/python3.8/site-packages
Requires: python-dateutil, jmespath, urllib3
Required-by: s3transfer, boto3


In [39]:
!pip show boto3

Name: boto3
Version: 1.37.10
Summary: The AWS SDK for Python
Home-page: https://github.com/boto/boto3
Author: Amazon Web Services
Author-email: None
License: Apache License 2.0
Location: /home/ubuntu/.local/lib/python3.8/site-packages
Requires: jmespath, s3transfer, botocore
Required-by: 


In [ ]:
boto3>=1.26.153 and botocore>=1.29.153.

In [ ]:
!pip install --upgrade mlflow boto3
# !pip install --upgrade
# !pip install --upgrade
# urllib3

Defaulting to user installation because normal site-packages is not writeable
Requirement already up-to-date: mlflow in ./.local/lib/python3.8/site-packages (2.17.2)
Requirement already up-to-date: boto3 in ./.local/lib/python3.8/site-packages (1.37.10)


In [54]:
!pip install requests "urllib3<2"

Defaulting to user installation because normal site-packages is not writeable
  Using cached urllib3-1.26.20-py2.py3-none-any.whl (144 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.2.3
    Uninstalling urllib3-2.2.3:
      Successfully uninstalled urllib3-2.2.3


In [22]:
!pip install --upgrade botocore

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 13.4 MB 2.4 MB/s eta 0:00:01
     |████████████████████████████████| 144 kB 66.6 MB/s eta 0:00:01
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.2.3
    Uninstalling urllib3-2.2.3:
      Successfully uninstalled urllib3-2.2.3
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

boto3 1.16.7 requires botocore<1.20.0,>=1.19.7, but you'll have botocore 1.37.10 which is incompatible.


In [18]:
!pip show urllib3

Name: urllib3
Version: 2.2.3
Summary: HTTP library with thread-safe connection pooling, file post, and more.
Home-page: None
Author: None
Author-email: Andrey Petrov <andrey.petrov@shazow.net>
License: None
Location: /home/ubuntu/.local/lib/python3.8/site-packages
Requires: 
Required-by: docker, requests, botocore


In [19]:
pip install --upgrade mlflow

Defaulting to user installation because normal site-packages is not writeable
Requirement already up-to-date: mlflow in ./.local/lib/python3.8/site-packages (2.17.2)
     |████████████████████████████████| 64 kB 529 kB/s eta 0:00:011


Note: you may need to restart the kernel to use updated packages.


In [ ]:
# import pandas as pd
# import numpy as np
# import pickle

# #from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, SparkTrials, Trials
# import mlflow
# from mlflow.tracking import MlflowClient

# import os
# from loguru import logger
# from functools import partial
# from argparse import ArgumentParser
# from catboost import CatBoostClassifier, Pool
# from sklearn.model_selection import cross_val_score, KFold
# from sklearn.metrics import roc_auc_score,fbeta_score,classification_report
# from sklearn.metrics import log_loss,roc_curve,auc,average_precision_score,accuracy_score,r2_score

# from catboost import CatBoostClassifier
# from sklearn.multiclass import OneVsRestClassifier
# from sklearn.preprocessing import MultiLabelBinarizer
# from sklearn.inspection import permutation_importance


# from pyspark.sql import SparkSession, DataFrame, functions as F
# from pyspark.sql.types import IntegerType,LongType,DoubleType,StringType

# from datetime import datetime, timedelta,timezone

# import bit_functions
# import config_btc
# import classes






def get_spark():

    spark = (
            SparkSession
            .builder
            .appName("OTUS")
            .getOrCreate()
            )
    return spark


def main(input_path):

    spark = get_spark()


    row_sdf = spark.read.parquet(input_path)

    #data_set_4h = pd.read_parquet('test_dataset_4H.parquet', engine='pyarrow')

    data_set_4h = row_sdf.toPandas()

    data_set_4h['target_new_up'] = np.where(data_set_4h['target_new']==1,1,0)
    data_set_4h['target_new_dw'] = np.where(data_set_4h['target_new']==-1,1,0)

    print('размер дата сета',data_set_4h.shape)

    cols_up_new = bit_functions.permutaion_list(data_set_4h,'target_new_up',clusters_selected_cols_08)

    cols_dw_new = bit_functions.permutaion_list(data_set_4h,'target_new_dw',clusters_selected_cols_08)


    final_columns_list = list(set(cols_up_new + cols_dw_new))

    print('длинна фичей финал',len(final_columns_list))


    X = data_set_4h[(data_set_4h.ROWNUM>=300)&(data_set_4h.ROWNUM<8500)][final_columns_list].copy()
    y = data_set_4h[(data_set_4h.ROWNUM>=300)&(data_set_4h.ROWNUM<8500)].target_new


    model =CatBoostClassifier(verbose=False)
#    model.set_params(**best_params)
    model_best = OneVsRestClassifier(estimator=model)
    model_best.fit(X,y)


if __name__ == "__main__":


    bucket_name = 'cold-s3-bucket'

    input_path = f"s3a://{bucket_name}/btc_prepaired_data.parquet"

    main(input_path)
